# JSON and Web APIs Tech Lab

Welcome to the JSON and APIs Tech Lab! In the code below, we'll walk through how to work with web APIs and JSON data using Python. 

## Making Web API Requests
Web API requests are made over HTTP via the use of formatted URLs. In daily usage, browsers like Chrome, Firefox, and Edge allow users to make HTTP requests to access files on the internet. In Python, the [`requests`](https://requests.readthedocs.io/en/master/) package is the preferred way to make HTTP requests and return their responses. 

`requests` is a third-party library that is installed by default with Anaconda distributions and can be imported using a simple import statement. 

In [ ]:
import requests

To make a GET request with `requests`, the `requests` package includes a simple [`.get()`](https://requests.readthedocs.io/en/master/api/#requests.get) function. The function takes one priamry argument, the URL you'd like to use in the request.

### Requests to Web Pages
Below, in very meta fashion, we use `requests` to view the `requests` documentation.

In [ ]:
r1 = requests.get("https://requests.readthedocs.io/en/master/")

The variable `r1` is now a [`Response`](https://requests.readthedocs.io/en/master/api/#requests.Response) object, which contains multiple useful attributes. Specifically, the `text` attribute stores the text of the response. In this case, since we are making a request to a web page, we expect the response to be HTML. 

The cell below shows the first 50 characters of the returned text.

In [ ]:
r1.text[:75]

Looking at the first couple characters of the returned text, we can see that the text is indeed HTML from the "<!DOCTYPE html" tag. 

Below, some nifty code attempts to render the HTML in a viewable format, although with some hiccups. 

In [ ]:
from IPython.core.display import  HTML
HTML(r1.text)

### Requests to Web APIs
Using Python making requests to web pages and rendering the HTML isn't exactly the best idea, especially given that browsers like Chrome and Safari are designed to do exactly that task. However, Python is great at interacting with web APIs!

Below, we use Python to make a request to the [Dog API](https://dog.ceo/dog-api/), a web API that shows random pictures of dogs from the [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/main.html). 

In [ ]:
#Gets a random image of a beagle
r2 = requests.get("https://dog.ceo/api/breed/beagle/images/random")

In [ ]:
#Print the text of the response
r2.text

## Working With JSON API Responses
Web APIs, unlike web pages, typically return data in JSON format, which stands for JavaScript Object Notation. For those familiar with Python, JSON objects can be thought of as the amalgamation of two Python data types - lists and dictionaries.

The Dog API response in a JSON object is essentially a Python dictionary with two key-value pairs. The `requests` module has a built-in method, called [`.json()`](https://requests.readthedocs.io/en/master/user/quickstart/#json-response-content) for converting JSON responses to Python data types.

In [ ]:
type(r2.json()) 

In [ ]:
r2.json()

Using the `.json()` method on the response object returns a Python dictionary, as we would expect. This dictionary can be used just the same as any Python dictionary. For example, we can use key indexing to get the URL of the picture of the dog returned by the API, as below.

In [ ]:
image_url = r2.json()["message"]
image_url

Using code similar to the code above used to render the HTML data, we can display that image in Jupyter.

In [ ]:
from IPython.core.display import Image
Image(image_url)

## Adding Parameters to API Requests
The request above to the Dog API did not include any parameters as it only required the API endpoint. Many web APIs, however, require parameters. 

Parameters come after the API endpoint and are separated from the URL of the endpoint by a question mark (?). Multiple parameters can be specified and parameters are separated from one another by an ampersand (&). 

### Example - The NPPES NPI Registry API

The request below is to a new endpoint, the [NPPES NPI Registry API](https://npiregistry.cms.hhs.gov/registry/help-api). This API allows users to search for physicians, nurses, dentists, and other medical providers via a unique ID called an NPI (National Provider Identifier). Users can also use the API to search for an NPI of a provider based on a set of criteria. 

Below, I use this web API to get information about a specific provider using their NPI. Two parameters are passed - the version (2.1) and the NPI of the physician (1972813608). 

In [ ]:
npi_r = requests.get("https://npiregistry.cms.hhs.gov/api/?version=2.1&number=1972813608")

### Now you try!

Use the `.json()` method of the variable `npi_r` to see more about this physician. 

### Adding Parameters with `requests`
The `requests` library provides a simpler way to pass parameters to web APIs rather than type out the ?s and &s yourself. You can store your parameters in a dictionary and then pass them as an argument in your request via the `params` named argument. 

Below, we make the same call to the NPPES NPI Registry API, but this time using the `params` argument. We can then see the full URL by printing the `.url` attribute of the response. Notice how the URL is the same as the one specified above!

In [ ]:
params = {"version" : "2.1", "number" : 1972813608}
npi_r = requests.get("https://npiregistry.cms.hhs.gov/api/", params=params)
npi_r.url

### Now You Try!
Use the `requests.get` method and the `params` argument to search the NPI registry using the `first_name` and `last_name` parameters, along with the `version` parameter as before. For the values of your parameters, search for a physician with the first name "Deborah" and the last name "Birx".

Store the response in a new variable called `npi_r2`. Then, visualize the JSON data by using the `.json()` method of the Response object.

The data above is very detailed. It includes a practitioner's NPI, name, address information, taxonomy, and more. However, the data isn't exactly tabular or "flat". Rather, there are multiple "levels" to the data, and some data points are 3 or 4 levels deep. 

## Getting Tabular Data from Web APIs

In order to work with more complex JSON data like that printed above, we will need to convert it to a tabular format. That can be done manually (i.e. by writing your own function to pull out the desired fields) or with help from Python packages. Before jumping in and understanding how to convert JSON data to a tabular format, let's do a quick Pandas refresher.

### Quick Pandas Refresher

Pandas is a Python package that allows Python users to work with tabular data. The main Pandas data type is called a DataFrame, which stores tabular data alongside column names and row indices. We can create a DataFrame using the DataFrame constructor function, `DataFrame()`. The function must have properly formatted data, such as a list of dictionaries which each indicate a row of data, or a dictionary of columns, where each key is a column and each value is all of that column's values. 

For example, the code below creates a dataframe with the columns "a", "b", and "c" (in that order) and whose rows are (1, 2, 3) and (4, 5, 6) .

In [ ]:
import pandas as pd #import Pandas
pd.options.display.max_columns = None #Make sure we can see all of the columns in Jupyter

In [ ]:
data = [{'a':1, 'b':2, 'c':3}, {'a':4, 'b':5, 'c':6}]
df = pd.DataFrame(data)
df.head()

### Using Pandas with the NPPPES NPI API
The data we recieved from the NPPES NPI (within `"results"`) is ordered as a list of dictionaries, perfect for converting to a tabular format using Pandas!

Let's see what happens if we convert the value of `"results"` to a Pandas DataFrame

In [ ]:
json_data = npi_r.json()
results = json_data["results"]
res_df = pd.DataFrame(results)
res_df.head()

It worked! We'll, kind of. The data that was only 1 level deep (like "number" and "enumeration_type") looks normal but the data that was deeper is still wrapped up. 

Pandas actually provides some ways to flatten data like this through a function called `json_normalize()`. Let's see what it happens if we use that method rather than the DataFrame constructor to create our dataset.

In [ ]:
pd.json_normalize(results)

We now see that the "basic" column has been split out into individual columns where each column has the "basic." prefix. However, this didn't impact the "addresses" or "taxonomies" fields. Why? That is because the "addresses" and "taxonomies" data are stored in lists/arrays, which `json_normalize` does not know how to flatten by default. To create the dataset we want, we'll need to write our own function.

### Writing Your Own Function to Pull Out Data
In the dataset above, the taxonomy data is stuck within a list of dictionaries under the column "taxonomies". Below is a function that parses the JSON data returned from `npi_r1` and returns a smaller, flattened dictionary for each result. The smaller dictionary includes data on the provider's NPI, their first name, last name, credential, and primary taxonomy description (the primary taxonomy is indicated by the value of "primary" in the taxonomy data being `True`.)

In [ ]:
#Define a function that returns a flattened dictionary
def flatten_json(jdata):
    flat_dict = {"number": jdata["number"]} #Get the NPI
    flat_dict.update(jdata["basic"]) #Get the "basic" info
    #Loop through taxonomies and pull primary taxonomy info
    for t in jdata["taxonomies"]:
        if t["primary"]:
            flat_dict.update(t)
    return flat_dict

In [ ]:
#Flatten all dictionaries in the results and create a DataFrame
fdata = [flatten_json(d) for d in npi_r.json()["results"]]
flat_df = pd.DataFrame(fdata)

In [ ]:
flat_df.head()

And there it is! Our JSON data is now in a tabular format that we could use to merge on to some existing data that we have for a future analysis. 

## Quiz! Putting It All Together
And that's about it! You now know how to make API requests using the `requests` library, how to convert the JSON data into Python data types and convert that data into a tabular format. 

With all of that knowledge, complete the following steps:
1. Use the same NPPES NPI API endpoint specified above and search for organizations that include the phrase "Charles River" by passing "charles river*" to the parameter "organization_name"
2. Get the JSON response from the NPPES 
3. Use the `flatten_json` function defined above to pull the NPI, basic organization information, and taxonomy info for each facility!


## Addendum: Authentication
Some Web APIs require users to register before use. These registration systems typically involve signing up online and creating an API key, which is then passed as an argument in each API call. 

Below, we created a login at [developer.nytimes.com](developer.nytimes.com) to use their [Books API](https://developer.nytimes.com/docs/books-product/1/overview), which gives rich access to book reviews and their Best Sellers list. Below, we make a call to the Books API and get the most recent Combined Print and E-Book NonFiction Best Sellers. Note how one of the parameters is an `api-key` which the NY Times provided after registration. To learn how to register for an NY Times API key of your own, click [here](https://developer.nytimes.com/get-started). 

In [ ]:
params = {"api-key" : "<your_api_key_here>", "list" : 'combined-print-and-e-book-nonfiction'}
nyt = requests.get("https://api.nytimes.com/svc/books/v3/lists.json", params=params)

In [ ]:
nyt.json()

In [ ]:
best_sellers = [r["book_details"][0]["title"] for r in nyt.json()["results"]]
best_sellers